In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import os
os.remove("bball.json")

In [3]:
class BSSpider(scrapy.Spider):
    name = "bballspy"
## Define the class
## Establish the online links you want to parse through
    start_urls = [
            'https://www.baseball-reference.com/teams/PHI/2008.shtml',
            'https://www.baseball-reference.com/teams/NYM/2008.shtml',
            'https://www.baseball-reference.com/teams/ATL/2008.shtml',
            'https://www.baseball-reference.com/teams/WAS/2008.shtml',
        ]
## Create a function look through the links via the XPATH link
    def parse(self, response):
        team=response.xpath('//*[@id="meta"]/div[2]/h1/span[2]/text()').get()
        table=response.xpath('//*[@id="team_batting"]')
        rows=table.xpath('tbody/tr')
        for r in range(len(rows)):
            
            yield { 
                'Team': team,
                'Name': rows[r].xpath('td[@data-stat="player"]/a/text()').get(),
                'Games': rows[r].xpath('td[@data-stat="G"]/text()').get(),
                'PA': rows[r].xpath('td[@data-stat="PA"]/text()').get(),
                'HR': rows[r].xpath('td[@data-stat="HR"]/text()').get(),
                'RBI': rows[r].xpath('td[@data-stat="RBI"]/text()').get(),
                'Hits': rows[r].xpath('td[@data-stat="H"]/text()').get(),
                }
    
# Tell the script how to run the crawler by passing in settings.
process = CrawlerProcess({
    'FEED_FORMAT': 'json',         # Store data in JSON format.
    'FEED_URI': 'bball.json',  # Name our storage file.
    'LOG_ENABLED': False           # Turn off logging for now.
})

# Start the crawler with our spider.
process.crawl(BSSpider)
process.start()
## Confirm it ran
print('Successful')

Successful


In [4]:
import pandas as pd
import json

BSdf=pd.read_json('bball.json', orient = 'records')
print(BSdf.shape)
print(BSdf.head())

(145, 7)
   Games    HR   Hits             Name     PA    RBI           Team
0  110.0   9.0   86.0  Brian Schneider  384.0   38.0  New York Mets
1  159.0  38.0  162.0   Carlos Delgado  686.0  115.0  New York Mets
2   87.0   3.0   73.0    Luis Castillo  359.0   28.0  New York Mets
3  159.0  16.0  204.0       Jose Reyes  763.0   68.0  New York Mets
4  160.0  33.0  189.0     David Wright  736.0  124.0  New York Mets


In [5]:
BSdf.sort_values('HR',ascending=False).head()

,Games,HR,Hits,Name,PA,RBI,Team
19,162.0,48.0,153.0,Ryan Howard,700.0,146.0,Philadelphia Phillies
1,159.0,38.0,162.0,Carlos Delgado,686.0,115.0,New York Mets
23,157.0,33.0,134.0,Pat Burrell,645.0,86.0,Philadelphia Phillies
4,160.0,33.0,189.0,David Wright,736.0,124.0,New York Mets
20,159.0,33.0,177.0,Chase Utley,707.0,104.0,Philadelphia Phillies


In [6]:
BSdf.sort_values('RBI',ascending=False).head()

,Games,HR,Hits,Name,PA,RBI,Team
19,162.0,48.0,153.0,Ryan Howard,700.0,146.0,Philadelphia Phillies
4,160.0,33.0,189.0,David Wright,736.0,124.0,New York Mets
1,159.0,38.0,162.0,Carlos Delgado,686.0,115.0,New York Mets
8,161.0,27.0,172.0,Carlos Beltran,706.0,112.0,New York Mets
20,159.0,33.0,177.0,Chase Utley,707.0,104.0,Philadelphia Phillies


In [7]:
BSdf.sort_values('Hits',ascending=False).head()

,Games,HR,Hits,Name,PA,RBI,Team
3,159.0,16.0,204.0,Jose Reyes,763.0,68.0,New York Mets
4,160.0,33.0,189.0,David Wright,736.0,124.0,New York Mets
20,159.0,33.0,177.0,Chase Utley,707.0,104.0,Philadelphia Phillies
8,161.0,27.0,172.0,Carlos Beltran,706.0,112.0,New York Mets
39,146.0,14.0,167.0,Shane Victorino,627.0,58.0,Philadelphia Phillies


In [9]:
BSdf['HR/PA']=BSdf.HR/BSdf.PA
BSdf.sort_values('HR/PA',ascending=False).head(15)

,Games,HR,Hits,Name,PA,RBI,Team,HR/PA
110,1.0,1.0,2.0,Lou Marson,4.0,2.0,Philadelphia Phillies,0.250000
112,4.0,1.0,2.0,Chris Snelling,4.0,1.0,Philadelphia Phillies,0.250000
97,16.0,2.0,5.0,Matt Stairs,19.0,5.0,Philadelphia Phillies,0.105263
19,162.0,48.0,153.0,Ryan Howard,700.0,146.0,Philadelphia Phillies,0.068571
1,159.0,38.0,162.0,Carlos Delgado,686.0,115.0,New York Mets,0.055394
23,157.0,33.0,134.0,Pat Burrell,645.0,86.0,Philadelphia Phillies,0.051163
40,134.0,24.0,114.0,Jayson Werth,482.0,67.0,Philadelphia Phillies,0.049793
20,159.0,33.0,177.0,Chase Utley,707.0,104.0,Philadelphia Phillies,0.046676
4,160.0,33.0,189.0,David Wright,736.0,124.0,New York Mets,0.044837
13,52.0,7.0,35.0,Ramon Castro,157.0,24.0,New York Mets,0.044586
